In [1]:
import pandas as pd
from config import OPENAI_Key
import openai
from openai import OpenAI
import json

openai.api_key = OPENAI_Key
OPENAI_API_KEY = OPENAI_Key
client = OpenAI(api_key=OPENAI_API_KEY)
import random

In [2]:
# # Helper function: get embeddings for a text
# def get_embeddings(text):
#     response = openai.Embedding.create(
#         model="text-embedding-ada-002",
#         input=text.replace("\n", " ")
#     )
#     embedding = response['data'][0]['embedding']
#     return embedding

# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# # Function to calculate cosine similarity
# def calculate_cosine_similarity(a, b):
#     return cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0][0]

# # Generate embeddings for text and labels
# def get_embeddings(texts):
#     response = openai.Embedding.create(
#         input=texts,
#         engine="text-embedding-ada-002",  # Choose an embeddings engine
#     )
#     return np.array([item['embedding'] for item in response['data']])

# # Your text to classify and possible labels
# text_to_classify = "I want to adopt a healthier lifestyle."
# labels = ["health and fitness", "entertainment", "education", "travel"]

# # Generate embeddings
# text_embedding = get_embeddings([text_to_classify])
# label_embeddings = get_embeddings(labels)

# # Calculate similarity and classify
# similarities = [calculate_cosine_similarity(text_embedding, label_emb) for label_emb in label_embeddings]
# classification_label = labels[np.argmax(similarities)]

# print(f"Text: '{text_to_classify}' is classified as '{classification_label}'")

Could also classify conditions into specific types temporal/spacial...


In [2]:
survey_questions  = pd.read_excel('Survey Questions Overview.xlsx', sheet_name='Wave 1', engine='openpyxl')
# Clean the survey questions dataframe to make it more usable for mapping
# Fill forward non-null ADICO Category values to apply them to all relevant rows
survey_questions_filled = survey_questions.ffill()

# Update the mapping to use a combination of 'Description' and 'Value labels' as the key
survey_questions_filled['Combined Key'] = survey_questions_filled['Description'].astype(str) + " - " + survey_questions_filled['Value labels'].astype(str)


# Since the dataset has multiple rows per question for different value labels, we'll create a unique mapping
# Create the new mapping dictionary
question_adico_mapping = survey_questions_filled.dropna(subset=['Description'])[['Combined Key', 'ADICO Category']].drop_duplicates().set_index('Combined Key')['ADICO Category'].to_dict()


# Filter out questions that are categorized as Attributes, Conditions, or Aims for clarity in analysis
attributes = [k for k, v in question_adico_mapping.items() if 'Attribute' in str(v) or 'Attribute/Condition' in str(v)]
conditions = [k for k, v in question_adico_mapping.items() if 'Condition' in str(v) or 'Aim/Condition' in str(v) or 'Attribute/Condition' in str(v)]
aims = [k for k, v in question_adico_mapping.items() if 'Aim' in str(v) or 'Aim/Condition' in str(v)]

In [7]:
survey_questions_filled

,ADICO Category,Question number\n (Questionnaire file),Variable Label\n (Data files),Description,Values,Value labels,Combined Key
0,NaN,ID,ID,Respondent ID to match respondents between waves,NaN,NaN,Respondent ID to match respondents between wav...
1,Attribute,not numbered,Q0_age,Age,1.0,16-24,Age - 16-24
2,Attribute,not numbered,Q0_age,Age,2.0,25-34,Age - 25-34
3,Attribute,not numbered,Q0_age,Age,3.0,35-44,Age - 35-44
4,Attribute,not numbered,Q0_age,Age,4.0,45-54,Age - 45-54
...,...,...,...,...,...,...,...
987,Condition,Q60a,Q60a_parent,Are you the parent or guardian of any children...,0.0,No,Are you the parent or guardian of any children...
988,Condition,Q60a,Q60a_parent,Are you the parent or guardian of any children...,1.0,Yes,Are you the parent or guardian of any children...
989,Condition,Q61,Q61_single_parent,Are you a single parent?,0.0,No,Are you a single parent? - No
990,Condition,Q61,Q61_single_parent,Are you a single parent?,1.0,Yes,Are you a single parent? - Yes


In [3]:
#For now we make a random selection of questions to identify the ADICO components but later we can provide the model with a predetermined selection
sample_questions_answers = random.sample(list(survey_questions_filled['Combined Key']),5)

print("Random questions: \n" + "\n".join(sample_questions_answers)+ "\n\n")

# Constructing the prompt
prompt = "For each of the question - answer combinations below identify if they can be used to extract an Attribute, Aim, or Condition:\n\n"
prompt += "\n".join(sample_questions_answers) + "\n\n"
prompt += "\n each question - answer pair can only be assigned to one of the three options"

#Explain to the GPT what their role is:
systemMessage = "You are a helpful assistant that reviews question - answer combinations from a survey and identifies if they can be used to define an Attribute, Aim, or Condition. You always output the same list of questions with their identified component(s) in JSON structure that will be read using the following: categorized_questions_json = json.loads(response.choices[0].message.content) and categorized_questions = pd.DataFrame(categorized_questions_json['questions'])."

#Give them some context information:
contentMessage = "An institutional statement refers to a structured representation of institutions using specific elements such as Attribute, Deontic, Aim, Condition, and Or Else (ADICO). These statements are used to define and understand the impacts, actions, and conditions associated with institutional rules, norms, and shared strategies within social systems.\n\n" 
contentMessage += "Attribute: Who is performing the aim action, e.g. demographics such as “Households in Indonesia” or “police officers”.\n\n" 
contentMessage += "Aim: The action performed by the entities with the listed attributes if all conditions are met, e.g. “reinforce their foundations bi-yearly” or “close the door”.\n\n"
contentMessage += "Condition: External factors that influence the demographic into performing the aim action, e.g. “if they live with more than four people” or “if the alarm goes off”\n\n"
contentMessage += "Combining these elements produces the following Institutional shared strategy: “Police officers (Attribute) lock the doors of their vehicle (Aim) if they leave the vehicle (Condition)”'\n\n"


response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": systemMessage},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": contentMessage},
  ]
)

# Parse JSON
categorized_questions_json = json.loads(response.choices[0].message.content)

# Convert JSON to dataframe
categorized_questions = pd.DataFrame(categorized_questions_json["questions"])

# Display dataframe
categorized_questions

Random questions: 
Raising the level of the ground floor above the most likely flood level - 1 - I am unable
What race do you self-identify as? (USA only) - Hispanic or Latino
Buying a spare power generator to power your home - I intend to implement this non-structural measure in the next 2 years
Buying a spare power generator to power your home - 2
Charitable organizations, NGOs - Yes




,question,identified_components
0,Raising the level of the ground floor above th...,[Aim]
1,What race do you self-identify as? (USA only),[Attribute]
2,Buying a spare power generator to power your home,[Aim]
3,Buying a spare power generator to power your h...,[]
4,"Charitable organizations, NGOs - Yes",[Attribute]


1st Successful Attempt:

Random questions: 

1. During times of hardship, my household can access the financial support I need (e.g. such as access to credit at a bank) - Attribute, Condition
2. Purchasing sandbags, or other water barriers - I intend to implement this non-structural measure in the future, after 2 years - Aim, Condition
3. My household can rely on the support from my government when I need help (e.g. receiving funding or support in the event of a natural disaster) - Attribute, Condition
4. What race do you self-identify as? (USA only) - Attribute
5. Keeping a working flashlight and/or a battery-operated radio and/or emergency kit in a convenient location - I intend to implement this non-structural measure in the next 12 months - Aim, Condition

In [4]:
# Random sample of questions for demonstration
sample_questions_answers = random.sample(list(survey_questions_filled['Combined Key']), 5)

# Simplified and integrated prompt
prompt = """You are a helpful assistant reviewing question-answer combinations from a survey to identify if they define an Attribute, Aim, or Condition based on the ADICO framework. 
An Attribute refers to demographics performing the aim action, e.g., “Households in Indonesia”. 
An Aim is the action performed if conditions are met, e.g., “reinforce foundations bi-yearly”. 
A Condition influences the action, e.g., “if they live with more than four people”.

Please always output the same list of questions with their identified component(s)  in JSON structure that will be read using the following: categorized_questions_json = json.loads(response.choices[0].message.content) and categorized_questions = pd.DataFrame(categorized_questions_json['questions']).

For each of the question-answer combinations below, identify them as an Attribute, Aim, or Condition:

""" + "\n".join(sample_questions_answers)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # Adjust engine as necessary
    messages=[
    {"role": "user", "content": prompt},
    ],
    temperature=0.5,
    max_tokens=250,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

# Parse JSON
categorized_questions_json = json.loads(response.choices[0].message.content)

# Convert JSON to dataframe
categorized_questions = pd.DataFrame(categorized_questions_json["questions"])

# Display dataframe
categorized_questions

,question,component
0,"Asking someone (local government, Civil Defens...",Aim
1,How many square meters is your accommodation? ...,Condition
2,Industry type,Attribute
3,Storing emergency food and water supplies,Aim
4,Industry type,Attribute


What about converting to ADICO Structure:

In [5]:
#For now we make a random selection of the ADICO components but later we can provide the model with a predetermined selection
selected_attributes = random.sample(attributes, 10)
selected_conditions = random.sample(conditions, 10)
selected_aims = random.sample(aims, 10)

# Constructing the prompt
prompt = "Generate Institutional shared strategy statements based on the following sets questions and answers containing Attributes, Aims and Conditions:\n\n"
prompt += "Attributes:\n" + "\n".join(selected_attributes) + "\n\n"
prompt += "Aims:\n" + "\n".join(selected_aims) + "\n\n"
prompt += "Conditions:\n" + "\n".join(selected_conditions) + "\n\n"

#Explain to the GPT what their role is:
systemMessage = "You are a helpful assistant that converts questions and answers from surveys into Institutional Behaviour Statements. You always output a list of statements that follow the following JSON structure: \n\n"
systemMessage += '{"statements":["Attribute":"People with {Attribute(s)}", "Aim":"do {Aim}", "Condition":"if {Condition(s)}]"”"\n\n"'
systemMessage += "Your output will be read using the following: converted_questions = json.loads(response.choices[0].message.content) and converted_questions = pd.DataFrame(converted_questions_json['statements'])."

#Give them some context information:
contentMessage = "An institutional statement refers to a structured representation of institutions using specific elements such as Attribute, Deontic, Aim, Condition, and Or Else (ADICO). These statements are used to define and understand the impacts, actions, and conditions associated with institutional rules, norms, and shared strategies within social systems.\n\n" 
contentMessage += "Attribute: Who or what the impacts of the institution apply to, e.g. “Households in Indonesia” or “police officers”.\n\n" 
contentMessage += "Aim: The definition of the impact that is applied or the action performed, e.g. “reinforce their foundations bi-yearly” or “close the door”.\n\n"
contentMessage += "Condition: the conditions that need to be satisfied in order for the aim to occur, e.g. “if they live with more than four people” or “if the alarm goes off”\n\n"
contentMessage += "Combining all these elements produces the following Institutional shared strategy: “Police officers (Attribute) lock the doors of their vehicle (Aim) if they leave the vehicle (Condition)”'\n\n"


response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": systemMessage},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": contentMessage},
  ]
)

# Parse JSON
converted_questions_json = json.loads(response.choices[0].message.content)

# Convert JSON to dataframe
converted_questions = pd.DataFrame(converted_questions_json["statements"])

# Display dataframe
converted_questions

,Attribute,Aim,Condition
0,People with the highest level of education as ...,trust social media completely from platforms l...,if they have implemented storing emergency foo...
1,"People in the Arts, entertainment, and recreat...",intend to move or store valuable assets on hig...,"if none of their friends, families, and neighb..."
2,People in the Natural resources industry (e.g....,believe that both the government and citizens ...,if they have missed one month of work


In [45]:
# Parse JSON
categorized_questions_json = json.loads(response.choices[0].message.content)

# Convert JSON to dataframe
categorized_questions = pd.DataFrame(categorized_questions_json["statements"])

# Display dataframe
categorized_questions

,Attribute,Aim,Condition
0,People with industry type as Retail,buy a spare power generator to power their home,if they intend to implement this non-structura...
1,People with highest level of education as Leve...,buy a spare power generator to power their home,if they intend to implement this non-structura...
2,"People with industry type as Public Sector, Go...",store or place important possessions in a mann...,if they intend to implement this non-structura...
3,People with highest level of education as High...,store or place important possessions in a mann...,if they intend to implement this non-structura...
4,People with industry type as Government and no...,raise the level of the ground floor above the ...,if they intend to implement this structural me...
5,People with highest level of education as Post...,raise the level of the ground floor above the ...,if they intend to implement this structural me...
6,People with industry type as Financial Service...,store emergency food and water supplies,if they have already implemented this non-stru...
7,People with highest level of education as Coll...,store emergency food and water supplies,if they have already implemented this non-stru...
8,"People with industry type as Engineering, Comp...",ask/petition government representatives to inc...,if they intend to implement this non-structura...


FIRST ATTEMPT:
'Institutional Behaviour Statements:\n\n
1. Retired individuals employed by private, for-profit companies in the natural resources industry (Attributes) intend to raise the level of the ground floor above the most likely flood level (Aim) if they plan to stay in their current house for 1-5 years (Condition).\n\n
2. Individuals with less than secondary education completed, working for other employers in the health or social care industry (Attributes) intend to strengthen their housing foundations to withstand water pressures (Aim) if they have the ability to undertake the structural measure in the next 6 months (Condition).\n\n
3. Individuals aged 35-44 working in the manufacturing industry of automobiles or automobile-related fields (Attributes) intend to install anti-backflow valves on pipes (Aim) if they plan to stay in their current residence for 1-5 years (Condition).\n\n
4. Individuals in the high school education category working in the retail industry (Attributes) intend to be active members of a community group aimed at making the community safer (Aim) if they have already implemented non-structural measures for emergency preparedness (Condition).'


SECOND:

1. People with a Villa accommodation category intend to install anti-backflow valves on pipes in the future, after 2 years, if they very frequently engage with social media. \n\n
2. People in the Construction industry intend to coordinate with neighbors in case of floods if not at home, within the next 6 months if more than five households have taken some adaptive action towards flooding.\n\n
3. Individuals with a 4-year university degree store emergency food and water supplies if they find it extremely ineffective to store them, and they are unable to store or place important possessions to avoid flood damage.\n\n
4. Students aim to implement the strategy of keeping a working flashlight and/or a battery-operated radio and/or emergency kit in a convenient location within the next 2 years if they do not trust social media at all.\n\n
5. Self-employed individuals engage with social media very frequently to take adaptive actions towards flooding if they intend to install a pump and/or drainage system to drain floodwater, finding it extremely effective.\n\n
6. People involved in heavy machinery or manufacturing industry types do not intend to implement the strategy of fixing water barriers if they are part-time workers (less than 8 hours a week) and find it extremely ineffective to store emergency food and water supplies.\n\n
7. Individuals in the real estate industry aim to store emergency food and water supplies within the next 6 months if they find it extremely ineffective and do not trust social media at all.\n\n
8. People with a university higher degree (Masters, MBA, PhD) do not trust social media at all and store emergency food and water supplies if they are unable to store or place important possessions to avoid flood damage.\n\n
9. Those involved in transport retail or wholesale industry types intend to move or store valuable assets on higher floors or elevated areas if it is not easy to leave their current place of residence, and they do not trust social media at all.\n\n
10. People working part-time (less than 8 hours a week) aim to implement the strategy of installing anti-backflow valves on pipes after 2 years if they do not find it easy to leave their current place of residence.'

In [50]:
selected_aims

['From social media (i.e. Facebook, Instagram, WeChat, Weibo, etc.) - 3',
 'Installing anti-backflow valves on pipes - I intend to implement this structural measure in the future, after 2 years',
 'Coordinating with the neighbors in case you are not home when a flood occurs, they would know what to do - I intend to implement this non-structural measure in the next 6 months',
 'From the general media - 2',
 'Storing emergency food and water supplies - I have already implemented this non-structural measure',
 'From the general media - 1 - Do not trust at all',
 'Keeping a working flashlight and/or a battery-operated radio and/or emergency kit in a convenient location - I intend to implement this non-structural measure in the next 2 years',
 'Storing emergency food and water supplies - I intend to implement this non-structural measure in the next 6 months',
 'Fixing water barriers (e.g. water-proof basement windows) - I do not intend to implement this structural measure',
 "In your opinio